# **Plants Disease Detection using Deep learning**

# Importing useful libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout,Flatten
from tensorflow.keras.applications import MobileNetV2, InceptionV3
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Loading original Plantvillage data set and storing image paths in a dataframe

In [ ]:
data_dir = '/kaggle/input/plantvillage-dataset/color'
class_folders = os.listdir(data_dir)
image_paths = []
labels = []

for class_folder in class_folders:
    class_path = os.path.join(data_dir, class_folder)
    image_files = os.listdir(class_path)
    for image_file in image_files:
        image_path = os.path.join(class_path, image_file)
        image_paths.append(image_path)
        labels.append(class_folder)

df = pd.DataFrame({'image_path': image_paths, 'label': labels})

In [ ]:
df.head()

# Show all classes contained in the data set 

In [ ]:
print("The classes:\n", np.unique(df['label']))

# Show number of samples per class

In [ ]:
class_counts = df['label'].value_counts()

plt.figure(figsize=(14, 8))
x = seaborn.barplot(x=class_counts.values, y=class_counts.index, orient='h')
plt.title('Class')
plt.xlabel('Number of Images')
plt.ylabel('Plant Types')
plt.tight_layout()  

for i, v in enumerate(class_counts.values):
    x.text(v + 5, i, str(v), color='black', va='center')

plt.show()

# Affichage d'un exemple d'image pour chaque classe

In [ ]:
num_classes = len(df['label'].unique())

num_images_per_row = 4
num_rows = (num_classes + num_images_per_row - 1) // num_images_per_row

plt.figure(figsize=(15, 5 * num_rows))  

for i, plant_class in enumerate(df['label'].unique()):
    plt.subplot(num_rows, num_images_per_row, i + 1)
    path = os.path.join(data_dir, df[df['label'] == plant_class]['image_path'].iloc[0])

    if os.path.exists(path):
        sample_image = cv2.imread(path)
        if sample_image is not None:
            plt.imshow(cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB))
            plt.title(plant_class)
            plt.axis('off')
        else:
            print(f"Error: Unable to load image from path: {path}")
    else:
        print(f"Error: Image path does not exist: {path}")

plt.tight_layout()
plt.show()

# Division du jeu de données en train , validation et test sets

In [ ]:
train_df_val_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df_val_df, test_size=0.2, random_state=42)

# Preprocessing des images de nos différents ensembles

In [ ]:
def load_images_for_cnn(train_df, val_df, test_df, batch_size=32, target_size=(224, 224)):
    
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        x_col='image_path',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    validation_generator = validation_datagen.flow_from_dataframe(
        val_df,
        x_col='image_path',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_datagen = ImageDataGenerator(rescale=1./255)

    test_generator = test_datagen.flow_from_dataframe(
        test_df,
        x_col='image_path',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, validation_generator, test_generator

# Résultats du preprocessing et nombre d'échantillons par ensemble

In [ ]:
train_generator, validation_generator,test_generator = load_images_for_cnn(train_df, val_df,
test_df, batch_size=32, target_size=(224, 224))

# Chargement du modèle MobileNetV2 original pour faire le transfer learning

In [ ]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.load_weights("/kaggle/input/mobilnetv2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1-0_224_no_top.h5")
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = Dropout(0.4)(x)
x= Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
learning_rate = 0.00001

# Transfer Learning to imporve accuracy
    
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
print('ok')

# Entrainement du modèle de transfer learning

In [ ]:
epochs = 3
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

In [ ]:
test_loss, test_accuracy= model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy : {test_accuracy}")

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Training and validation accuracy after transfer learning << MobilNetV2  >>")

plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title("Training and validation loss after transfer learning << MobilNetV2 >>")

plt.legend()
plt.show()

In [ ]:
test_predictions = model.predict(test_generator)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_true_labels = test_generator.classes

In [ ]:
#precision
precision = precision_score(test_true_labels, test_predicted_labels, average='macro')

# recall
recall = recall_score(test_true_labels, test_predicted_labels, average='macro')

# F1-score
f1 = f1_score(test_true_labels, test_predicted_labels, average='macro')

print('precision : ', precision ,'\nRecall : ', recall , '\nF1-mesure: ', f1)

In [ ]:
model.save('Plant_classification_model_MobileNetV2.h5')

# **Fine-tuning** du modèle issu de l'entrainement précédent

In [ ]:

base_model_1 = load_model('Plant_classification_model_MobileNetV2.h5')

# model = Model(inputs=base_model.input, outputs=predictions)
learning_rate = 0.0001

# Transfer Learning to imporve accuracy
# Unfreezing layers in the MobileNetV2 model
# By unfreezing the last 40 layers in the MobileNetV2 model, 
# we are allowing those layers to adapt to this specific dataset 

for layer in base_model_1.layers[-30:]:
    layer.trainable = True
    

fine_tuned_model = base_model_1    
fine_tuned_model.compile(optimizer=RMSprop(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
print('ok')

#  Entrainement du modèle fine-tuné sur nos données

In [ ]:
epochs = 5
history_f =fine_tuned_model.fit(train_generator, epochs=epochs, validation_data=validation_generator)



# Evaluation sur le jeu de test

In [ ]:
test_loss, test_accuracy= fine_tuned_model.evaluate(test_generator)
print(f"Test Loss : {test_loss}, Test Accuracy : {test_accuracy}")

# Courbe d'évolution de l'accuracy et loss pendant l'entrainement

In [ ]:
# Plot training and validation accuracy
plt.plot(history_f.history['accuracy'], label='Training Accuracy')
plt.plot(history_f.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Training and validation accuracy for fine-tuning << MobilNetV2 >>")

plt.show()

# Plot training and validation loss
plt.plot(history_f.history['loss'], label='Training Loss')
plt.plot(history_f.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title("Training and validation loss for fine-tuning << MobilNetV2 >>")

plt.legend()
plt.show()

# Prédiction des classes par le modèle sur le jeu de test

In [ ]:
test_predictions = fine_tuned_model.predict(test_generator)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_true_labels = test_generator.classes

# Matrice de confusion

In [ ]:
plt.figure(figsize=(18, 12))
conf_matrix = confusion_matrix(test_true_labels, test_predicted_labels)
class_names = [label for label in np.unique(df['label'])]
seaborn.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Evaluation du modèle sur le jeu de test

In [ ]:
#precision
precision = precision_score(test_true_labels, test_predicted_labels, average='macro')

# recall
recall = recall_score(test_true_labels, test_predicted_labels, average='macro')

# F1-score
f1 = f1_score(test_true_labels, test_predicted_labels, average='macro')

print('precision: ', precision ,'\nRecall : ', recall , '\nF1-mesure : ', f1)

In [ ]:
labels = {
    0: 'Apple___Apple_scab',
    1: 'Apple___Black_rot',
    2: 'Apple___Cedar_apple_rust',
    3: 'Apple___healthy',
    4: 'Blueberry___healthy',
    5: 'Cherry_(including_sour)___Powdery_mildew',
    6: 'Cherry_(including_sour)___healthy',
    7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
    8: 'Corn_(maize)___Common_rust_',
    9: 'Corn_(maize)___Northern_Leaf_Blight',
    10: 'Corn_(maize)___healthy',
    11: 'Grape___Black_rot',
    12: 'Grape___Esca_(Black_Measles)',
    13: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    14: 'Grape___healthy',
    15: 'Orange___Haunglongbing_(Citrus_greening)',
    16: 'Peach___Bacterial_spot',
    17: 'Peach___healthy',
    18: 'Pepper,_bell___Bacterial_spot',
    19: 'Pepper,_bell___healthy',
    20: 'Potato___Early_blight',
    21: 'Potato___Late_blight',
    22: 'Potato___healthy',
    23: 'Raspberry___healthy',
    24: 'Soybean___healthy',
    25: 'Squash___Powdery_mildew',
    26: 'Strawberry___Leaf_scorch',
    27: 'Strawberry___healthy',
    28: 'Tomato___Bacterial_spot',
    29: 'Tomato___Early_blight',
    30: 'Tomato___Late_blight',
    31: 'Tomato___Leaf_Mold',
    32: 'Tomato___Septoria_leaf_spot',
    33: 'Tomato___Spider_mites Two-spotted_spider_mite',
    34: 'Tomato___Target_Spot',
    35: 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    36: 'Tomato___Tomato_mosaic_virus',
    37: 'Tomato___healthy'
}

# Sauvegarde du modéle

In [ ]:
fine_tuned_model.save('Plant_classification_model_MobileNetV2_fine_tuned.h5')

# 

# **Inception V3 architcture**

# 

# Chargement du modèle

In [ ]:
# InceptionV3
base_model_V3 = InceptionV3(weights='imagenet', include_top = False, input_shape=(224, 224, 3))
base_model_V3.load_weights("/kaggle/input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5")

for layer in base_model_V3.layers:
    layer.trainable = False

x = base_model_V3.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model_2 = Model(inputs=base_model_V3.input, outputs=predictions)

learning_rate = 0.0001

model_2.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrainement(transfer learning) 

In [ ]:
epochs = 3
history_2 = model_2.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Evaluation sur le test set

In [ ]:
test_loss, test_accuracy = model_2.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Courbe d'évolution de l'accuracy et loss pendant l'entrainement

In [ ]:
# Plot training and validation accuracy
plt.plot(history_2.history['accuracy'], label='Training Accuracy')
plt.plot(history_2.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Training and validation accuracy << InceptionV3 >>")

plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history_2.history['loss'], label='Training Loss')
plt.plot(history_2.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title("Training and validation loss << InceptionV3 >>")

plt.legend()
plt.show()


# Prédiction des classes par le modèle sur le jeu de test

In [ ]:
test_predictions = model_2.predict(test_generator)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_true_labels = test_generator.classes



# Evaluation du modèle sur le jeu de test

In [ ]:
#precision
precision = precision_score(test_true_labels, test_predicted_labels, average='macro')

# recall
recall = recall_score(test_true_labels, test_predicted_labels, average='macro')

# F1-score
f1 = f1_score(test_true_labels, test_predicted_labels, average='macro')

print('precision : ', precision ,'\nRecall : ', recall, '\nF1-mesure : ', f1)

# Sauvegarde du modèle

In [ ]:
model_2.save('Plant_classification_model_InceptionV3.h5')

# Fine-tuning sur le modèle précédent

In [ ]:

base_model_V3_f = load_model('Plant_classification_model_InceptionV3.h5')

# model = Model(inputs=base_model.input, outputs=predictions)
learning_rate = 0.0001

# Transfer Learning to imporve accuracy
# Unfreezing layers in theInceptionV3 model
# By unfreezing the last 30 layers in the InceptionV3 model, 
# we are allowing those layers to adapt to this specific dataset 

for layer in base_model_V3_f.layers[-30:]:
    layer.trainable = True
    

fine_tuned_model_2 = base_model_V3_f    
fine_tuned_model_2.compile(optimizer=RMSprop(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
print('ok')

# Entrainement du modèle fine-tuné

In [ ]:
epochs = 5
history_ft =fine_tuned_model_2.fit(train_generator, epochs=epochs, validation_data=validation_generator)



# Evaluation sur le jeu de test

In [ ]:
test_loss, test_accuracy= fine_tuned_model_2.evaluate(test_generator)
print(f"Test Loss : {test_loss}, Test Accuracy : {test_accuracy}")

#  Courbe d'évolution de l'accuracy et loss pendant l'entrainement

In [ ]:
# Plot training and validation accuracy
plt.plot(history_ft.history['accuracy'], label='Training Accuracy')
plt.plot(history_ft.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Training and validation accuracy << InceptionV3 >>")

plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history_ft.history['loss'], label='Training Loss')
plt.plot(history_ft.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title("Training and validation loss << InceptionV3 >>")

plt.legend()
plt.show()


# Prédiction des classes par le modèle sur le jeu de test

In [ ]:
test_predictions = fine_tuned_model_2.predict(test_generator)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_true_labels = test_generator.classes


# Matrice de confusion

In [ ]:
plt.figure(figsize=(18, 12))
conf_matrix_ = confusion_matrix(test_true_labels, test_predicted_labels)
class_names = [label for label in np.unique(df['label'])]
seaborn.heatmap(conf_matrix_, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix ')
plt.show()

# Evaluation du modèle sur le jeu de test

In [ ]:
#precision
precision = precision_score(test_true_labels, test_predicted_labels, average='macro')

# recall
recall = recall_score(test_true_labels, test_predicted_labels, average='macro')

# F1-score
f1 = f1_score(test_true_labels, test_predicted_labels, average='macro')

print('precision : ', precision ,'\nRecall : ', recall, '\nF1-mesure : ', f1)

# Sauvegarde du modèle

In [ ]:
fine_tuned_model_2.save('Plant_classification_model_InceptionV3_fine_tuned.h5')

In [ ]:
MobileNetV2_ft= load_model('Plant_classification_model_MobileNetV2_fine_tuned.h5')

In [ ]:
MobileNetV2_ft_test_loss, MobileNetV2_ft_test_accuracy = MobileNetV2_ft.evaluate(test_generator)
print(f"Test Loss: {MobileNetV2_ft_test_loss}, Test Accuracy: {MobileNetV2_ft_test_accuracy}")

In [ ]:
InceptionV3_ft= load_model('Plant_classification_model_InceptionV3_fine_tuned.h5')

In [ ]:
InceptionV3_ft_test_loss, InceptionV3_ft_test_accuracy = InceptionV3_ft.evaluate(test_generator)
print(f"Test Loss: {InceptionV3_ft_test_loss}, Test Accuracy: {InceptionV3_ft_test_accuracy}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

models = ("MobileNetV2_ft", "InceptionV3_ft")
test_results = {
    'Test Accuracy': (MobileNetV2_ft_test_accuracy*100, InceptionV3_ft_test_accuracy*100),
    'Test Loss': (MobileNetV2_ft_test_loss*100, InceptionV3_ft_test_loss*100)
}

x = np.arange(len(models))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in test_results.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Accuracy (%)')
ax.set_title('Plant Disease Detection Models')
ax.set_xticks(x + width, models)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 250)

plt.show()